In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#参考URL
#https://www.kaggle.com/shaitender/light-gbm-digit-recognizer-way-10-lines-tutorial

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# --- Load & build data ---
print('Loading data...')

train=pd.read_csv('../input/digit-recognizer/train.csv')
print(train.shape)
test=pd.read_csv('../input/digit-recognizer/test.csv')
print(train.shape)

train.head(5)

In [ ]:
#4行目、1列目のlabelが"4"なので、4を取得してみる
four = train.iloc[3,1:]
four.shape

In [ ]:
four= four.values.reshape(28,28)
plt.imshow(four,cmap='gray')

In [ ]:
#visuallise the array
print(four[5:-5,5:-5])

In [ ]:
#おなじみのdescribe

#avearage values/distributions of features
description = train.describe()
description

In [ ]:
num_class = len(train.iloc[:,0].unique())

In [ ]:
#yは目的変数なので、labelのデータだけ取り出す
x_train= train.iloc[:,1:]
y_train= train.iloc[:,0]

x_test = test.values
y_test= test.iloc[:,0]

#rescaling the feature
from sklearn.preprocessing import scale
x_train = scale(x_train)
x_test=scale(x_test)

#print

#xは説明変数
#yは目的変数
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

print(y_train.head(10))

In [ ]:
import lightgbm as lgb
print ('Training lightgbm')

lgtrain = lgb.Dataset(x_train, y_train)
lgval = lgb.Dataset(x_test, y_test)

# params multiclass
params = {
          "objective" : "multiclass",           
          "max_depth": -1,
           "num_class":num_class,
          "learning_rate" : 0.0001,                 
          "verbosity" : 1 }

model = lgb.train(params, lgtrain, 500, valid_sets=[lgtrain, lgval], early_stopping_rounds=750, verbose_eval=200)

In [ ]:
# predict results
results = model.predict(x_test)

# select the index's with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)

In [ ]:
submission.head()